# data merge

In [1]:
import json
import os
import pandas as pd
from typing import Dict, List
import re
import pickle

# 강남구 - mango

In [2]:
gangnam = pd.read_csv('./Seoul/강남구.csv')
gangnam[gangnam['도로명전체주소'].isna()]
gangnam['소재지전체주소']=gangnam['소재지전체주소'].fillna('서울특별시 강남구 수서동 214-2')

In [4]:
mango_1 = json.load(open('./output/seoul/mango/강남구_mango_add.json'))
mango_2 = json.load(open('./output/seoul/mango/강남구_further_output_mango_score.json'))

## index consistency

In [19]:
gangnam_further = pd.read_csv('./output/seoul/mango/강남구_further.csv',index_col=0)

In [22]:
gangnam_further

,Unnamed: 0,사업장명,소재지전체주소,도로명전체주소,자치구,query
0,0,커스텀커피,서울특별시 강남구 세곡동 598,"서울특별시 강남구 헌릉로569길 35, 지상1층 101호 (세곡동)",강남구,서울 강남구 커스텀커피
1,2,코코버블티 강남역점,서울특별시 강남구 역삼동 817-11,"서울특별시 강남구 강남대로96길 9, 지상1층 (역삼동)",강남구,서울 강남구 코코버블티
2,10,씨유 역삼형지점,서울특별시 강남구 역삼동 797-16 명호빌딩,"서울특별시 강남구 논현로 327, 명호빌딩 지상1층 (역삼동)",강남구,서울 강남구 씨유
3,16,행복한 김치,서울특별시 강남구 삼성동 159-7 현대백화점,"서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)",강남구,서울 강남구 행복한 김치
4,18,GS25 논현동우점,서울특별시 강남구 논현동 232-5,"서울특별시 강남구 봉은사로37길 13, 지상1층 (논현동)",강남구,서울 강남구 GS25
...,...,...,...,...,...,...
1320,4015,CU 신사드림점,서울특별시 강남구 신사동 526번지 두인빌딩,"서울특별시 강남구 압구정로4길 10, 두인빌딩 지하1층 1호 (신사동)",강남구,서울 강남구 CU
1321,4035,CU강남신동아점,서울특별시 강남구 수서동 736번지 신동아아파트,"서울특별시 강남구 광평로47길 17, 상가동 지상1층 104호,105호 (수서동, ...",강남구,서울 강남구 CU강남신동아점
1322,4037,아마스빈강남2호점,서울특별시 강남구 역삼동 818-8번지,"서울특별시 강남구 강남대로94길 21, 1층 103호 (역삼동)",강남구,서울 강남구 아마스빈강남2호점
1323,4046,스타벅스 논현역사거리점,서울특별시 강남구 논현동 142-2 JS타워 지상1층,"서울특별시 강남구 강남대로 538, JS타워 지상1층 (논현동)",강남구,서울 강남구 스타벅스


In [27]:
mango_for_check = {j['name']:j for j in mango_1.values()}

## crawling 안된 index 파악

In [28]:
mango_crawling_index = []
mango_output = {}
for i,j in mango_for_check.items():
    # 사업장명이 2개라면 도로명 주소 기준으로 
    if len(gangnam.index[gangnam['사업장명']==i])==1:
        idx = gangnam.index[gangnam['사업장명']==i][0]
        mango_output[idx] = j.get('mango_score',[])
        mango_crawling_index.append(idx)
    else:
        indice = gangnam.index[gangnam['사업장명']==i]
        gold_idx = None
        for idx in indice:
            # na 값인 지 확인
            if type(gangnam['소재지전체주소'][idx])==float:
                if gangnam['도로명전체주소'][idx].strip() == j['도로명전체주소'].strip():
                    gold_idx = idx
                    break
                else:
                    continue
            else:
                if gangnam['소재지전체주소'][idx].strip() == j['소재지전체주소'].strip():
                    gold_idx = idx
                    break
                else:
                    continue
        assert gold_idx is not None, (i,j)
        mango_output[gold_idx]=j.get('mango_score',[])
        mango_crawling_index.append(idx)

In [29]:
for i,j in mango_2.items():
    mango_output[gangnam_further.loc[int(i),'Unnamed: 0']]=j

In [38]:
pickle.dump(mango_output,open('./output/seoul/강남구/mango_output.pkl','wb'))

# 강남구 - 카카오

In [13]:
kakao = json.load(open('./output/seoul/강남구/kakao.json'))
kakao_further = json.load(open('./output/seoul/kakao/강남구_output_kakao_score.json'))
kakao_further_output = json.load(open('./output/seoul/kakao/강남구_output.json'))
gangnam_further = pd.read_csv('./output/seoul/kakao/강남구.csv',index_col=0)

In [5]:
kakao_for_check = {i:j for i,j in kakao.items()}

In [6]:
kakao_crawling_index = []
kakao_output = {}
for i,j in kakao_for_check.items():
    # 사업장명이 2개라면 도로명 주소 기준으로 
    if len(gangnam.index[gangnam['사업장명']==i])==1:
        idx = gangnam.index[gangnam['사업장명']==i][0]
        kakao_output[idx] = j.get('kakao_score',[])
        kakao_crawling_index.append(idx)
    else:
        indice = gangnam.index[gangnam['사업장명']==i]
        gold_idx = None
        for idx in indice:
            # na 값인 지 확인
            if type(gangnam['소재지전체주소'][idx])==float:
                if gangnam['도로명전체주소'][idx].strip() == j['도로명전체주소'].strip():
                    gold_idx = idx
                    break
                else:
                    continue
            else:
                if gangnam['소재지전체주소'][idx].strip() == j['소재지전체주소'].strip():
                    gold_idx = idx
                    break
                else:
                    continue
        assert gold_idx is not None, (i,j)
        kakao_output[gold_idx]=j.get('kakao_score',[])
        kakao_crawling_index.append(idx)

In [11]:
c = 0
for i,j in kakao.items():
    if not j['kakao_score']:
        print(i)
        print(j)
        input()
        c+=1

송오브송즈로스터즈
{'소재지전체주소': '서울특별시 강남구 신사동 569-8 압구정 디자인빌딩 ', '도로명전체주소': '서울특별시 강남구 논현로163길 17, 압구정 디자인빌딩 지상1층 (신사동)', '자치구': '강남구', 'query': '서울특별시 강남구 신사동 송오브송즈로스터즈', 'kakao_score': []}


KT그룹희망나눔재단
{'소재지전체주소': '서울특별시 강남구 수서동 734 KT ', '도로명전체주소': '서울특별시 강남구 광평로47길 28, KT 지상6층 (수서동)', '자치구': '강남구', 'query': '서울특별시 강남구 수서동 KT그룹희망나눔재단', 'kakao_score': []}


새콤달콤
{'소재지전체주소': '서울특별시 강남구 대치동 911-13 ', '도로명전체주소': '서울특별시 강남구 역삼로 452, 지상1층 108호 (대치동)', '자치구': '강남구', 'query': '서울특별시 강남구 대치동 새콤달콤', 'kakao_score': []}


머그샷(Mug,shot)
{'소재지전체주소': '서울특별시 강남구 대치동 936-3 ', '도로명전체주소': '서울특별시 강남구 선릉로64길 8-2, 지상1층 101호 (대치동)', '자치구': '강남구', 'query': '서울특별시 강남구 대치동 머그샷(Mug,shot)', 'kakao_score': []}


카우커피(cow coffee)
{'소재지전체주소': '서울특별시 강남구 청담동 7-12 ', '도로명전체주소': '서울특별시 강남구 삼성로145길 14, 지상1층 (청담동)', '자치구': '강남구', 'query': '서울특별시 강남구 청담동 카우커피(cow coffee)', 'kakao_score': []}


에그슬럿 강남점
{'소재지전체주소': '서울특별시 강남구 역삼동 626-30 ', '도로명전체주소': '서울특별시 강남구 봉은사로20길 54, 지상2층 10,11호 (역삼동)', '자치구': '강남구', 'query': '서울특별시 강남구 역삼동 에그슬럿 강남점', 'kakao_score': []}


에버그릭 역삼점
{'소재지전체주소': '서울특별시 강남구 역삼동 736-24 엘지에클라트 ', '도로명전체주소': '서울특별시 강남구 테헤란로20길 19, 엘지에클라트 지하1층 B101호 (역삼동)', '자치구': '강남구', 'query': '서울특별시 강남구 역삼동 에버그릭 역삼점', 'kakao_score': []}


KeyboardInterrupt: Interrupted by user

In [10]:
c

3959

In [7]:
kakao_output

{0: [{'name': '커스텀커피 세곡점',
   'address': '서울 강남구 헌릉로569길 35 1층 101호',
   'score': '3.3',
   'reviews': ['스텔라\n후기\n63\n별점평균\n3.1\n2022.12.10.\n추천포인트\n맛\n친절\n흑임자라떼!!!\n꼭드세요\n좋아요\n메뉴 더보기',
    'Jm\n후기\n301\n별점평균\n3.0\n2022.09.18.\n영수증을 달라고 하면 혐오스러운 표정으로 바라보는 사장이 기자리고 있을 거임. 아마 영수증은 후기조작하는 지인들 전용인듯. 영수증 달라고 하는 자를 잠재적 비판리뷰어로 추측하는 한심한 마인드때문에 망할듯..\n좋아요\n메뉴 더보기',
    'PTLEE\n후기\n33\n별점평균\n3.1\n2022.08.17.\n말 그대로 인생 라떼 네요\n근래 먹은것중 젤 맛있어요\n좋아요\n메뉴 더보기']}],
 1: [],
 2: [{'name': '코코버블티 강남역점',
   'address': '서울 강남구 강남대로96길 9 1층',
   'score': '1.0',
   'reviews': ['룰루\n후기\n2\n별점평균\n1.0\n2022.11.18.\n진짜 너무너무너무너무너무맛이없음 살면서 이렇게 맛없는 버블티는 처음\n좋아요\n메뉴 더보기',
    '맑음\n후기\n23\n별점평균\n2.6\n2022.08.10.\n여자알바 오픈했냐고 물어보니 단답에 핸드폰만 보고있어요\n좋아요\n메뉴 더보기']}],
 3: [],
 4: [{'name': '에스프레소엔투샷',
   'address': '서울 강남구 언주로135길 32-3 1층',
   'score': '5.0',
   'reviews': ['Oai\n후기\n52\n별점평균\n4.3\n2022.09.21.\n추천포인트\n맛\n가성비\n친절\n분위기\n근처에서 찍은 사진\n라떼 착한 가격에 \n착하지 않은 고급진 원두맛,,\n분위기까지 완벽 ..\n또오고싶어요 !!\n좋아요\n메뉴 더보기',
    'kh\n후기\

In [49]:
for i,j in kakao_further.items():
    kakao_output[gangnam_further.loc[int(i),'Unnamed: 0']]=j

In [51]:
pickle.dump(kakao_output,open('./output/seoul/강남구/kakao_output.pkl','wb'))